### Get Data from GitHub

In [2]:
!git clone https://github.com/ahmedramadan96/EALPR.git

Cloning into 'EALPR'...
remote: Enumerating objects: 18740, done.
remote: Counting objects: 100% (12906/12906), done.
remote: Compressing objects: 100% (12903/12903), done.
remote: Total 18740 (delta 12), reused 12892 (delta 3), pack-reused 5834
Receiving objects: 100% (18740/18740), 273.19 MiB | 14.77 MiB/s, done.
Resolving deltas: 100% (32/32), done.
Updating files: 100% (18735/18735), done.


### Main Functions

In [3]:
def getFilename(filename):
    filename_image =filename.split('/')[4].split('.')[0]+'.png'
    if len(find_files(filename_image,'./EALPR/EALPR- Plates dataset'))==0:
        filename_image =filename.split('/')[4].split('.')[0]+'.jpg'
    if len(find_files(filename_image,'./EALPR/EALPR- Plates dataset'))==0:
        filename_image =filename.split('/')[4].split('.')[0]+'.JPG'
    if len(find_files(filename_image,'./EALPR/EALPR- Plates dataset'))==0:
        filename_image =filename.split('/')[4].split('.')[0]+'.jpeg'
    filepath_image = os.path.join('./EALPR/EALPR- Plates dataset',filename_image)
    return filepath_image


In [3]:
def img_new_pound(img_resized, x1p,x2p,y1p,y2p):
    h,w= img_resized.shape
    x1 = x1p * w;
    x2 = x2p * w;
    y1 = y1p * h;
    y2 = y2p * h;
    return x1,x2,y1,y2

In [4]:
def img_pound_persent(img_orig, x1, x2, y1, y2):
    h,w= img_orig.shape
    x1p = x1 / w;
    x2p = x2 / w;
    y1p = y1 / h;
    y2p = y2 / h;
    return x1p,x2p,y1p,y2p

In [4]:
def find_files(filename, search_path):
    result = []
# Waking top-down from the root
    for root, dir, files in os.walk(search_path):
         if filename in files:
                result.append(os.path.join(root, filename))
    return result

In [6]:
# A function that takes in an image and a bounding box and returns the percentage of the bounding box in the image.

def get_persent_of_each_pound(img, pounding):
        h,w= img.shape
        xmin, ymin,xmax, ymax, = pounding
        xmin_persent = xmin/w
        xmax_persent= xmax/w
        ymin_persent= ymin/h
        ymax_persent = ymax/h
        return xmin_persent, xmax_persent,ymin_persent,ymax_persent


In [7]:
def get_new_pound_after_resize_img(img,pounding_persents):
        h, w= img.shape
        xmin_persent, ymin_persent, xmax_persent, ymax_persent= pounding_persents
        xmax = xmax_persent*w
        ymax = ymax_persent*h
        xmin = xmin_persent*w
        ymin = ymin_persent*h
        return xmin, xmax, ymin, ymax


In [5]:
def convert(bx,by,bw,bh):
    xmx=((2*w*bx+w*bw)/2)
    xmn=(2*w*bx-xmx)
    ymx=((2*h*by+h*bh)/2)
    ymn=(2*h*by-ymx)
    return xmn,xmx,ymn,ymx ,h,w

In [9]:
# def unconvert(xmin,xmax,ymin,ymax):
#     bx =( xmax+xmin)/(2*w)
#     by =(ymax+ymin)/(2*h)
#     bh= ((2*ymax) -(2*h*by))/h
#     bw= ((2*ymax) -(2*w*bx))/w
#     return bx, by,bw,bh
    

### importing main libaries

In [6]:
import os
import plotly.express as px 
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
labels_path='./EALPR/EALPR- LP characters dataset/Characters Labeling'
img_path='./EALPR/EALPR- LP characters dataset/Characters'

In [91]:
existing_labels = []
y_hat = []
image_label=[]
for file in os.listdir(labels_path):
    if file=='classes.txt':
        continue
    file_name = labels_path+'/'+file
    existing_labels.append(file_name)
    y_2d = []

    with open(file_name) as cur:
            info = cur.readlines()
            classes_num = [0 for i in range(27)]
            for i in info:  # str
                    tmp = i.split()  # list of i
                    classes_num[int(tmp[0])]=1
                    for i in range(len(tmp)):
                        tmp[i] = float(tmp[i])
                    y_2d.append(list(tmp[1:]))  # 2d list (7,5) of the labels
#                     y_2d.append(list(tmp))  # 2d list (7,5) of the labels
            image_label.append(list(classes_num))

#                         new V

            for i in range(8):
                    if len(y_2d) <=7 :
#                         if float(i) not in classes_num:
    #                         y_2d.append([float(i), 0.00, 0.00, 0.00, 0.00])
                            y_2d.append([ 0.00, 0.00, 0.00, 0.00])
                    else:
                        break
                    
            cur.close()

    # one 2d array for each image and then 3d for all images
    y_hat.append(y_2d)

#     print(y_hat)
#     print('__'*100)


In [93]:
Y=y_hat
len(image_label),len(Y)
image_label[0]

[0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1]

In [14]:
images_paths=[]
for file in existing_labels:
    images_paths.append(getFilename(file))
print(len(images_paths))
images_paths[:10]#random check

1983


['./EALPR/EALPR- Plates dataset/2051_license_plate_1.png',
 './EALPR/EALPR- Plates dataset/0086_license_plate_1.png',
 './EALPR/EALPR- Plates dataset/0738_license_plate_1.png',
 './EALPR/EALPR- Plates dataset/1809_license_plate_1.png',
 './EALPR/EALPR- Plates dataset/1302_license_plate_1.png',
 './EALPR/EALPR- Plates dataset/0237_license_plate_1.png',
 './EALPR/EALPR- Plates dataset/0734_license_plate_1.png',
 './EALPR/EALPR- Plates dataset/0968_license_plate_1.png',
 './EALPR/EALPR- Plates dataset/1642_license_plate_1.png',
 './EALPR/EALPR- Plates dataset/1477_license_plate_1.png']

In [15]:
Y[0]

[[0.18686868686868688,
  0.7018348623853211,
  0.10101010101010101,
  0.3944954128440367],
 [0.29797979797979796,
  0.7018348623853211,
  0.1111111111111111,
  0.3761467889908257],
 [0.4065656565656566,
  0.6880733944954128,
  0.09595959595959595,
  0.3853211009174312],
 [0.8762626262626263,
  0.6238532110091743,
  0.12626262626262627,
  0.3853211009174312],
 [0.7449494949494949,
  0.6467889908256881,
  0.11616161616161616,
  0.26605504587155965],
 [0.601010101010101,
  0.6697247706422018,
  0.13131313131313133,
  0.25688073394495414],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0]]

In [16]:
file_path = images_paths[0] 

img = cv2.imread(file_path ,0)
img = cv2.resize(img,(500,500))
h,w = img.shape
fig = px.imshow(img)
fig.update_layout(width=500, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Example')
for i in range(8):
    xmn,xmx,ymn,ymx,_,_=convert(float(Y[0][i][0]),float(Y[0][i][1]),float(Y[0][i][2]),float(Y[0][i][3]))
    print(f'{xmn} {xmx} {ymn} {ymx} {h} {w}')
    fig.add_shape(type='rect',x0=xmn, x1=xmx, y0=ymn, y1=ymx, xref='x', yref='y',line_color='red')
fig

68.18181818181819 118.68686868686868 252.29357798165137 449.54128440366975 500 500
121.21212121212122 176.76767676767676 256.88073394495416 444.95412844036696 500 500
179.2929292929293 227.27272727272728 247.70642201834858 440.3669724770642 500 500
406.5656565656566 469.69696969696975 215.59633027522938 408.256880733945 500 500
343.4343434343434 401.5151515151515 256.88073394495416 389.9082568807339 500 500
267.67676767676767 333.33333333333337 270.64220183486236 399.0825688073395 500 500
0.0 0.0 0.0 0.0 500 500
0.0 0.0 0.0 0.0 500 500


In [17]:
# data = []
# output = []
# for ind in range(len(images_paths)):
#     image = images_paths[ind]
#     each_img_label = []
#     img_arr = cv2.imread(image, 0)  # image to array of numbers
#     h, w = img_arr.shape
#     # Prepprocesing
#     # resize to the standard vompitable size of the pre-trained transfer learning model
#     img = cv2.resize(img_arr,(250,250))
#     img=img/255.0
#     # load the image as an array after resizing
# #     load_image_arr = img_to_array(load_image)
# #     norm_load_image_arr = load_image_arr/255.0
# #     Normalization

#     # Normalization to labels
#     for i in range(8):
#         bx, by, bw, bh = Y[ind][i]
#         h, w = img_arr.shape

#         xmin, xmax, ymin, ymax, _, _ = convert(bx, by, bw, bh)
#         xmin, xmax, ymin, ymax = img_pound_persent(img_arr, xmin, xmax, ymin, ymax)
#         xmin, xmax, ymin, ymax = img_new_pound(img, xmin, xmax, ymin, ymax)
#         pounding = [xmin, xmax, ymin, ymax]
#         h, w = img.shape

#         nxmin, nxmax = xmin/w, xmax/w
#         nymin, nymax = ymin/h,ymax/h
#         label_norm = [nxmin, nxmax, nymin, nymax]  # Normalized output
#         each_img_label.append(label_norm)
#     output.append(each_img_label)  # y
#     # Append
#     data.append(img)  # X

In [ ]:
# img=images_paths[0]
# output=[]
# img_arr =cv2.imread(img,0)
# print(img_arr.shape)
# img = cv2.resize(img_arr,(250,250))
# for i in range(7):
#         bx,by,bw,bh = Y[3][i]
#         h,w=img_arr.shape
#         xmin1,xmax1,ymin1,ymax1,_,_=convert(bx,by,bw,bh)
# #         pounding =[xmin,xmax,ymin,ymax]
#         x1p,x2p,y1p,y2p=img_pound_persent(img_arr,xmin1,xmax1,ymin1,ymax1)
#         xmin,xmax,ymin,ymax=img_new_pound(img,x1p,x2p,y1p,y2p)
# #         xmin_persent,xmax_persent,ymin_persent,ymax_persent=get_persent_of_each_pound(img_arr,pounding)
# #         pounding=[xmin_persent,xmax_persent,ymin_persent,ymax_persent]
# #         xmin,xmax,ymin,ymax= get_new_pound_after_resize_img(img,pounding)
# #         print(f'{xmin},{xmax},{ymin},{ymax}')
# #         print('_'*25)
# #         print(f'{xmin1},{xmax1},{ymin1},{ymax1}')
# #         print('.*.'*25)
# #         print(f'{x1p},{x2p},{y1p},{y2p}')
#         print(ymin,ymax,w,h)
#         h,w=img.shape
#         bx,by,bw,bh=unconvert(xmin,xmax,ymin,ymax)
#         print(bx, by,bw,bh ,h,w)
#         label_norm = [bx, by,bw,bh]  # Normalized output
#         output.append(label_norm)

In [ ]:
# # print(np.max(output))
# for i in output:
#     print(i)

In [ ]:
# def show_img_after_change_size(image,Y):    
# #     img = array_to_img(image, scale=True) #returns PIL Image
# #     img = img.resize((150, 150)) #resize image
# #     arr = img_to_array(img) #convert back to array
# #     img = cv2.imread(img)
#     #     h,w,s = img.shape
#     file_path = images_paths[0] 

#     img = cv2.imread(file_path)
#     img = cv2.resize(img,(250,250))
#     n=0
#     fig = px.imshow(img)
#     fig.update_layout(width=224, height=224, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Example')
#     for i in range(7):
#         if Y[i][0]==0.0:
#             break
#         xmn,xmx,ymn,ymx ,h ,w=convert(Y[i][0],(Y[i][1]),(Y[i][2]),(Y[i][3]))
#         fig.add_shape(type='rect',x0=xmn, x1=xmx, y0=ymn, y1=ymx, xref='x', yref='y',line_color='red')
#         print(Y[i][0],(Y[i][1]),(Y[i][2]),(Y[i][3]))
#         print(f'{xmn} {xmx} {ymn} {ymx} {h} {w}')
#         print('___'*22)
#     return fig
# # n=X[0]*255

# show_img_after_change_size(data[0]*255,output)

In [ ]:
# show_img_after_change_size(data[0]*255,Y)

In [55]:
X=[]
for i in range(len(images_paths)):
    image = images_paths[i]
    img_arr =cv2.imread(image,0)
    h,w= img_arr.shape
    img_arr= cv2.resize(img_arr,(224,224))
    X.append(img_arr)
X= np.array(X,dtype=np.float32)/255.0


In [19]:
y = np.array(Y)
y = y.reshape(-1 ,8*4)

In [100]:
image_label= np.array(image_label)
num_classes = 27
# Split the data into training and testing set using sklearn.
label_train, label_val, label_test=image_label[:1427],image_label[1427:159+1427],image_label[1427+159:]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.1,random_state=0)
X_train.shape,X_val.shape ,X_test.shape,y_train.shape,y_val.shape,y_test.shape

((1427, 224, 224),
 (159, 224, 224),
 (397, 224, 224),
 (1427, 32),
 (159, 32),
 (397, 32))

In [95]:
image_label[0]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1])

In [103]:
X_train = X_train.reshape(-1, 224,224, 1)
X_val = X_val.reshape(-1, 224,224, 1)
X_test = X_test.reshape(-1, 224,224, 1)

In [106]:
input_shape = (224,224,1)
input_layer = tf.keras.layers.Input(input_shape)

#create the base layers
# base_layers = layers.experimental.preprocessing.Rescaling(1./255, name='bl_1')(input_layer)
base_layers = layers.Conv2D(16, 3, padding='same', activation='relu', name='bl_2')(input_layer)
base_layers = layers.MaxPooling2D(name='bl_3')(base_layers)
base_layers = layers.Conv2D(32, 3, padding='same', activation='relu', name='bl_4')(base_layers)
base_layers = layers.MaxPooling2D(name='bl_5')(base_layers)
base_layers = layers.Conv2D(64, 3, padding='same', activation='relu', name='bl_6')(base_layers)
base_layers = layers.MaxPooling2D(name='bl_7')(base_layers)
base_layers = layers.Flatten(name='bl_8')(base_layers)
#create the classifier branch
classifier_branch = layers.Dense(128, activation='relu', name='cl_1')(base_layers)
classifier_branch = layers.Dense(num_classes, name='cl_head')(classifier_branch)  
#create the localiser branch
locator_branch = layers.Dense(512, activation='relu', name='bb_0')(base_layers)
locator_branch = layers.Dense(256, activation='relu', name='bb_1')(base_layers)

locator_branch = layers.Dense(128, activation='relu', name='bb_2')(locator_branch)
locator_branch = layers.Dense(64, activation='relu', name='bb_3')(locator_branch)
locator_branch = layers.Dense(32, activation='sigmoid', name='bb_head')(locator_branch)
model = tf.keras.Model(input_layer,
           outputs=[classifier_branch,locator_branch])

losses = {"cl_head":tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
   "bb_head":tf.keras.losses.MSE}
model.compile(loss=losses, optimizer='Adam', metrics=['accuracy'])
trainTargets = {
    "cl_head": label_train,
    "bb_head": y_train
}
validationTargets = {
    "cl_head": label_val,
    "bb_head": y_val
}


In [108]:
for layer in cnn.layers:
    if layer.name.startswith('bl_'):
        layer.trainable = False
        
for layer in cnn.layers:
    if layer.name.startswith('bb_'):
        layer.trainable = False

In [109]:
cnn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 98, 58, 512)       5120      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 49, 29, 512)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 47, 27, 256)       1179904   
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 23, 13, 256)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 21, 11, 128)       295040    
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 10, 5, 128)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 8, 3, 64)         

In [ ]:
history = model.fit(X_train, trainTargets,
             validation_data=(X_val, validationTargets),
             batch_size=4,
             epochs=20,
             shuffle=True,
             verbose=1)

In [114]:
cnn = keras.Sequential([
        #     layer input
#             layers.InputLayer(input_shape=(224, 224, 1)),

            # data augmentation
#            tf.keras.layers.RandomContrast(0.1),
        #     layers.RandomFlip("horizontal_and_vertical"),
        #     layers.RandomRotation(0.1),
        #     layers.RandomTranslation(0.2,0.1),
        #     layers.RandomZoom(0.3),
        #     cnn layers
 layers.Conv2D(512, 3,input_shape=(224,224,1),  activation='relu'),
      layers.MaxPooling2D(),

  layers.Conv2D(256, (3,3),  activation='relu'),
  layers.MaxPooling2D((2,2)),
  layers.Conv2D(128, (3,3), activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, (3,3), activation='relu'),
  layers.MaxPooling2D(),
    layers.Dropout(0.2),
    # dense
    layers.Flatten(),

    layers.Dense(2084,activation='relu'),
    layers.Dense(1024,activation='relu'),
    layers.Dropout(0.15),

    layers.Dense(512,activation='relu',name='D1'),
    layers.Dense(400, activation='relu'),

    layers.Dense(256, activation='relu',name='D2'),
    layers.Dense(128, activation='relu',name='cl_head'), 
    layers.Dense(64, activation='relu',name='D4'),
    layers.Dense(8*4,activation='sigmoid',name='bb_head'),
    
        ])

cnn.summary()
        


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 222, 222, 512)     5120      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 111, 111, 512)     0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 109, 109, 256)     1179904   
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 54, 54, 256)       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 52, 52, 128)       295040    
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 24, 24, 64)       

In [113]:
losses = {"cl_head":tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
   "bb_head":tf.keras.losses.MSE}
model.compile(loss=losses, optimizer='Adam', metrics=['accuracy'])
trainTargets = {
    "cl_head": label_train,
    "bb_head": y_train
}
validationTargets = {
    "cl_head": label_val,
    "bb_head": y_val
}


In [115]:
cnn.compile(
            optimizer= keras.optimizers.Adam(0.001),
            loss=keras.losses.MSE,
            metrics=['accuracy']
        )


In [ ]:
cnn.fit(
    X_train,y_train,
      validation_data=(X_val,y_val),
    epochs=140,
                 shuffle=True,
      batch_size=32,

#             callbacks = [tensorboard_callback], 
          )
# history = cnn.fit(train_images, trainTargets,
#              validation_data=(validation_images, validationTargets),
#              batch_size=4,
#              epochs=training_epochs,
#              verbose=1)

Epoch 1/140
45/45 [==============================] - 17s 338ms/step - loss: 0.0117 - accuracy: 0.0932 - val_loss: 0.0136 - val_accuracy: 0.1384
Epoch 2/140
45/45 [==============================] - 12s 265ms/step - loss: 0.0112 - accuracy: 0.1114 - val_loss: 0.0151 - val_accuracy: 0.0818
Epoch 3/140
45/45 [==============================] - 12s 265ms/step - loss: 0.0110 - accuracy: 0.1507 - val_loss: 0.0142 - val_accuracy: 0.0755
Epoch 4/140
45/45 [==============================] - 12s 265ms/step - loss: 0.0099 - accuracy: 0.1668 - val_loss: 0.0133 - val_accuracy: 0.1447
Epoch 5/140
45/45 [==============================] - 12s 265ms/step - loss: 0.0094 - accuracy: 0.2102 - val_loss: 0.0146 - val_accuracy: 0.1635
Epoch 6/140
45/45 [==============================] - 12s 265ms/step - loss: 0.0089 - accuracy: 0.2698 - val_loss: 0.0145 - val_accuracy: 0.1509
Epoch 7/140
45/45 [==============================] - 12s 265ms/step - loss: 0.0080 - accuracy: 0.3224 - val_loss: 0.0144 - val_accuracy:

In [47]:
cnn.fit(
    X_train,y_train,
      validation_data=(X_val,y_val),
    epochs=140,
      shuffle=True,
#             callbacks = [tensorboard_callback], 
          )

Epoch 1/140
45/45 [==============================] - 2s 40ms/step - loss: 0.0010 - accuracy: 0.8970 - val_loss: 0.0207 - val_accuracy: 0.3019
Epoch 2/140
45/45 [==============================] - 2s 38ms/step - loss: 0.0025 - accuracy: 0.8038 - val_loss: 0.0215 - val_accuracy: 0.2642
Epoch 3/140
45/45 [==============================] - 2s 38ms/step - loss: 0.0037 - accuracy: 0.7043 - val_loss: 0.0213 - val_accuracy: 0.2893
Epoch 4/140
45/45 [==============================] - 2s 38ms/step - loss: 0.0027 - accuracy: 0.7589 - val_loss: 0.0212 - val_accuracy: 0.3019
Epoch 5/140
45/45 [==============================] - 2s 38ms/step - loss: 0.0022 - accuracy: 0.8059 - val_loss: 0.0211 - val_accuracy: 0.3019
Epoch 6/140
45/45 [==============================] - 2s 38ms/step - loss: 0.0019 - accuracy: 0.8108 - val_loss: 0.0202 - val_accuracy: 0.3019
Epoch 7/140
45/45 [==============================] - 2s 41ms/step - loss: 0.0018 - accuracy: 0.8332 - val_loss: 0.0193 - val_accuracy: 0.2956
Epoch 

In [48]:
cnn.evaluate(X_test,y_test)

13/13 [==============================] - 0s 31ms/step - loss: 0.0205 - accuracy: 0.2771


[0.020548222586512566, 0.2770780920982361]

### NOTES:
        stil working on it not finsihed
        problem : OVERFIT